In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)
import plotly
import plotly.express as px
import numpy as np

In [ ]:
# %run scripts/question_and_optimal_answer.py

In [ ]:
df.columns

In [ ]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))
df = dataset.dataframe

In [ ]:
cols = ['box_x', 'pipe_x', 'enemy_speed', 'mario_speed']
fig = go.Figure()
fig = make_subplots(rows=1, cols=4)
for i, c in enumerate(cols):
    fig.add_trace(go.Histogram(x=df[c].values), row=1, col=i+1)
    fig.update_xaxes(title_text=c, row=1, col=i+1)

# Overlay both histograms
fig.update_layout(barmode='overlay', showlegend=False, title_text="Hidden state & question distributions")
# Reduce opacity to see both histograms
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_traces(opacity=0.5)
fig.show()


In [ ]:
cols = ['answer_box', 'answer_pipe', 'answer_enemy']
fig = go.Figure()
for c in cols:
    fig.add_trace(go.Histogram(x=df[c].values, name=c))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_layout(title_text="Optimal answer distributions")
fig.update_xaxes(title_text="Optimal answer value")
fig.update_yaxes(title_text="Frequency")
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
fig = px.scatter_3d(df, x="box_x", y="mario_speed", z="answer_box")
fig.show()

In [ ]:
fig = px.scatter_3d(df, x="enemy_speed", y="mario_speed", z="answer_enemy")
fig.show()